In [1]:
%cd ..

/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3


/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from augments import WikipediaCrawler
from prompts import load_template
from utils import DataVersionManager

/data/ephemeral/gamchan/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
dvm = DataVersionManager()
wc = WikipediaCrawler()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
ANSWER = "answer"
REASONING = "reasoning"

In [ ]:
# 현재 실험은 1.2.x 입니다.
df = dvm.search_latest_experiments_data(1, 1, False)
df.head()

,Unnamed: 0,id,paragraph,question,choices,answer,question_plus,reasoning
0,0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...
1,1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)..."
2,2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...
3,3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...
4,4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="wikipedia_search_keyword_with_reasoning.txt", template_type="keyword_extraction")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

keyword_extractor = prompt | llm

In [ ]:
def extract_keyword(data: pd.Series) -> list[str]:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        list[str]: 추출된 5개 키워드
    """
    response = keyword_extractor.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
            "answer": data[ANSWER],
            "reasoning": data[REASONING],
        },
    )
    response_split = response.content.split(",")
    keywords = [keyword.strip() for keyword in response_split]
    return keywords


tqdm.pandas()
df["keywords"] = df.progress_apply(extract_keyword, axis=1)
df.head()

100%|██████████| 1627/1627 [25:10<00:00,  1.08it/s]


,Unnamed: 0,id,paragraph,question,choices,answer,question_plus,reasoning,keywords
0,0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,"[송준길, 서인, 예송논쟁, 인조반정, 탕평 정치]"
1,1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...","[삼한, 고려사, 동녕부, 몽골, 지눌]"
2,2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,"[소정방, 김문영, 신라, 김춘추, 백제]"
3,3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,"[영조, 서원, 대전회통, 비변사, 통상 수교]"
4,4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,"[대무예, 고구려, 발해, 3성 6부, 중앙집권]"


In [ ]:
df.to_csv("temp.csv", index=False)  # 1차 중간 결과

추출된 키워드가 위키피디아에 존재하는지 여부 확인

- 키워드 추출 결과가 유효한지 확인하기 위한 작업입니다.


In [ ]:
# 키워드를 유용하게 다루기 위해 키워드 별로 칼럼으로 분리
keywords_expanded = pd.DataFrame(df["keywords"].tolist(), index=df.index)
keywords_expanded.columns = [f"keyword_{i+1}" for i in range(keywords_expanded.shape[1])]
new_df = pd.concat([df.drop(columns=["keywords"]), keywords_expanded], axis=1)

In [ ]:
# 위키피디아 페이지가 있는지 키워드 별로 확인
for col in keywords_expanded.columns:
    new_df[f"{col}_exists"] = pd.Series()
    new_df[f"{col}_page"] = pd.Series()

for idx, row in tqdm(new_df.iterrows()):
    for col in keywords_expanded.columns:
        result = wc.is_exists(row[col])
        new_df.loc[idx, f"{col}_exists"] = result[0]
        new_df.loc[idx, f"{col}_page"] = result[1] if result[0] else None

new_df.head()

1627it [1:18:34,  2.90s/it]


,Unnamed: 0,id,paragraph,question,choices,answer,question_plus,reasoning,keyword_1,keyword_2,...,keyword_1_exists,keyword_1_page,keyword_2_exists,keyword_2_page,keyword_3_exists,keyword_3_page,keyword_4_exists,keyword_4_page,keyword_5_exists,keyword_5_page
0,0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...,문제 해결 과정\n\n배경 분석\n지문은 송준길이 상복의 절차에 대해 논한 내용을 ...,송준길,서인,...,True,"송준길(宋浚吉, 1606년 12월 28일 ~ 1672년 12월 2일)은 조선시대 후...",True,"서인(西人)은 조선 시대 붕당정치 초기의 당파로, 1575년(선조 8)의 동서분당(...",True,"예송(禮訟, 문화어: 례송) 또는 예송논쟁(禮訟論爭, 문화어: 례송론쟁)은 예절에 ...",True,"인조반정(仁祖反正)은 1623년 4월 11일(음력 3월 12일) 김류, 이귀, 신경...",False,None
1,1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN,"문제 해결 과정\n\n배경 분석\n주어진 지문은 고려사에서 발췌된 내용으로, (가)...",삼한,고려사,...,True,"삼한(三韓)은 삼국 시대 이전, 곧 원삼국 시대에 한반도에 자리 잡고 있던 마한(馬...",True,고려사(高麗史)는 조선 전기에 편찬된 고려 왕조의 역사를 저술한 역사서이며 기전체(...,True,동녕부(東寧府)는 원나라가 고려 서경을 편입한 후 설치한 통치기관이다.\n\n역사\...,True,"몽골국(몽골어: Монгол Улс, ᠮᠣᠩᠭᠣᠯᠤᠯᠤᠰ 멍걸 올스 듣기 , 음차:...",True,"지눌(知訥, 1158년~1210년 4월 22일(음력 3월 27일))은 고려 중기 ~..."
2,2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN,문제 해결 과정\n\n배경 분석\n지문은 **소정방**과 신라군의 상황을 다루고 있...,소정방,김문영,...,True,"소정방(蘇定方, 592년 ~ 667년)은 당나라의 장군이다. 소열(蘇烈)이 본명이지...",True,"김문영(金文穎, 생몰년 미상)은 7세기 신라의 장군이자 관인이었다. 김문영(金文永)...",True,"신라(新羅)는 기원전 57년부터 기원후 935년까지 고구려, 백제와 함께 고대 한반...",True,"태종 무열왕(太宗武烈王, 603년 ~ 661년 음력 6월)은 신라(新羅)의 제29대...",True,"백제(百濟)는 한국의 고대 국가 중 하나로, 고구려, 신라와 함께 삼국 시대를 구성..."
3,3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN,문제 해결 과정\n\n배경 분석\n지문은 (가) 인물이 대궐 앞에 모인 선비들에 의...,영조,서원,...,True,"영조(英祖, 1694년 10월 31일(음력 9월 13일) ~ 1776년 4월 22일...",True,서원(書院)는 중국에서 발단 한자 문화권의 동아시아 (일본 한국 베트남)에 파급된...,True,《대전회통》(大典會通)은 1865년(고종 2년)에 조두순(趙斗淳) 등이 왕명에 따라...,True,비변사(備邊司)는 조선시대 군국기무(軍國機務)를 관장한 문무합의기구(文武合議機構)다...,False,None
4,4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN,문제 해결 과정\n\n배경 분석\n주어진 지문은 대무예(대무신왕)가 일본에 보낸 국...,대무예,고구려,...,True,"무왕(武王, ?~737년)은 발해의 제2대 왕이다. 휘는 무예(武藝)이다. 연호는 ...",True,고구려(高句麗)는 한국의 고대 왕조국가 중 하나이다. 국성은 고씨(高氏)이다. 본래...,True,발해(渤海) 또는 발해국(渤海國)은 고구려 멸망 이후 고구려 계승을 표방하던 대조영...,True,삼성육부제(三省六部制)는 과거 중세 중국 왕조의 중앙 정치 제도이다. 이는 주변국에...,True,"중앙집권(中央集權, 영어: centralisation, centralization)..."


In [ ]:
# _exists 열만 추출
exists_columns = [col for col in new_df.columns if col.endswith("_exists")]

true_count = new_df[exists_columns].sum().sum()  # True 값의 총합
false_count = (new_df[exists_columns] == False).sum().sum()  # False 값의 총합

true_count, false_count

(4235, 3900)

In [ ]:
new_df.to_csv(dvm.update_file_path(1, 2, "micro", is_experiment=False, save_in_experiment=True), index=False)